In [59]:
import pandas as pd
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

## Análise e cálculo de similaridades de artigos a partir do algoritmo K-NN
Utilizaremos três abordagens:
* Utilizar os títulos dos artigos como entrada, e para a realização do cálculo de similaridades relativas a outros títulos
* Utilizar os títulos dos artigos como entrada, e para a realização do cálculo de similaridades relativas a outros textos (conteúdo do artigo)
* Utilizar os textos tanto para a entrada quanto para o cálculo de similaridades relativas a outros textos
  
Ao fim desse notebook, responderemos as seguintes perguntas:

1. Os vizinhos são os mesmos em todos os casos?

2. Se não, quais foram diferentes? 

3. Olhando a sátira de entrada, qual das configurações acima (1,2 ou 3) você acha que trouxe os melhores vizinhos e por quê?

4. Nos casos que houve diferença, por que você acha que isso aconteceu?

### K-NN: títulos como entrada e para cálculo de similaridades

In [60]:
df = pd.read_csv("https://raw.githubusercontent.com/caiolibanio/atividade_NLP/master/csv_satiras_politicas.csv")
df.head(20)

,title,text,label
0,crise e tao grande que nem tiozao do pave fez piada na noite de natal,a familia guimaraes passou a noite de natal perplexa. genival o tiozao do pave deles desde 1985...,satire
1,nao me representam diz jesus sobre intolerantes que apedrejaram crianca que saia do candomble,uma menina de 11 anos apedrejada ao sair de uma festa de candomble no ultimo domingo no rio de j...,satire
2,marina silva e heloisa helena montam novo partido o partido esquerdista da republica unida p. e...,insatisfeitas com seus partidos com as siglas dos partidos com os politicos com as tardes ensola...,satire
3,dez propostas que podem realmente mudar o brasil,o instituto nupal nucleo de pesquisas da america latina fez uma pesquisa que mostrou ideias que ...,satire
4,apresentadora do cidade alerta bahia dara curso de como ser otimista e feliz,assassinatos sequestros mortes violentas. nenhuma noticia parece abalar a apresentadora do cida...,satire
5,onu aponta brasil como pioneiro no combate ao futuro,no dia do trabalhador uma noticia especial para os brasileiros a organizacao das nacoes unidas o...,satire
6,se quem pergunta por que o povo nao esta na rua fosse para a rua elas estariam cheias diz espec...,por que o povo nao esta nas ruas como e que depois de tudo que vem acontecendo esta todo mundo t...,satire
7,iemanja revela que nao tem mais onde guardar barquinhos e pede criatividade dos devotos,iemanja esta preocupada com a proximidade do ano novo. segundo ela nem o oceano inteiro esta se...,satire
8,em universidade alunos perseguem professores de matematica usando video policial como justifica...,apos o caso de perseguicao e morte do traficante matematico que vem repercutido muito na televis...,satire
9,chefe de milicia vaidoso cria um monumento em sua homenagem,o lider da milicia de rio das pedras alessandro assuncao conhecido como xerife ergueu uma estatu...,satire


In [61]:
# Os títulos escolhidos são os seguintes:

index1 = 9
index2 = 101
index3 = 63

print("{df1_title1};\n{df1_title2};\n{df1_title3}".format(df1_title1 = df.iloc[index1].title, df1_title2 = df.iloc[index2].title, df1_title3 = df.iloc[index3].title))

 chefe de milicia vaidoso cria um monumento em sua homenagem;
 cego agradece a deus por nao poder ver outdoor contra privilegios para deficientes;
 tristeza no orgulho hetero cancer de prostata podera ser detectado com exame de urina


In [62]:
df_titles = df.drop(columns=['text', 'label'])

pd.options.display.max_colwidth = 100
titles_series = df_titles['title']
titles_series

0                                     crise e tao grande que nem tiozao do pave fez piada na noite de natal
1             nao me representam diz jesus sobre intolerantes que apedrejaram crianca que saia do candomble
2        marina silva e heloisa helena montam novo partido o partido esquerdista da republica unida p. e...
3                                                          dez propostas que podem realmente mudar o brasil
4                              apresentadora do cidade alerta bahia dara curso de como ser otimista e feliz
                                                       ...                                                 
4369                               pesquisa 85 das pessoas ja abandonaram a dieta iniciada no novo ano novo
4370                19h de sexta ja e hora de admitir a derrota e ligar para o peguete de sempre diz estudo
4371                    senado aprova hoje nova reforma ortografica com uso do sinal de acentuacao de ironi
4372                        

In [63]:
def buildTFIDFVectorizer(docs):
    vect = TfidfVectorizer()
    matrix = vect.fit_transform(docs) 
    return matrix, vect

def docSimilarities(doc, matrix):
    res = matrix.dot(doc.transpose())
    return res

In [64]:
tfidf_matrix, tfidf_vect = buildTFIDFVectorizer(titles_series)
tfidf_matrix.shape # São 4374 linhas (documentos) e 9747 colunas (palavras do vocabulário);

(4374, 9747)

In [65]:
query_doc_title1 = tfidf_matrix.getrow(index1)
query_doc_title2 = tfidf_matrix.getrow(index2)
query_doc_title3 = tfidf_matrix.getrow(index3)

titles_doc_sim1 = np.array(docSimilarities(query_doc_title1, tfidf_matrix).toarray())
titles_doc_sim2 = np.array(docSimilarities(query_doc_title2, tfidf_matrix).toarray())
titles_doc_sim3 = np.array(docSimilarities(query_doc_title3, tfidf_matrix).toarray())

# titles_doc_sim1 = np.array(titles_doc_sim1.toarray())
# titles_doc_sim1 = np.array(titles_doc_sim1.toarray())
# titles_doc_sim1 = np.array(titles_doc_sim1.toarray())

titles_doc_sim1

array([[0.00608894],
       [0.        ],
       [0.        ],
       ...,
       [0.01153888],
       [0.        ],
       [0.00700184]])

In [66]:
titles_doc_sim1.shape == titles_doc_sim2.shape == titles_doc_sim3.shape

True

In [67]:
titles_sorted_indexes1 = np.argsort(titles_doc_sim1, axis=None)
titles_sorted_indexes2 = np.argsort(titles_doc_sim2, axis=None)
titles_sorted_indexes3 = np.argsort(titles_doc_sim3, axis=None)

most_similar_title1 = titles_sorted_indexes1[-2]
second_most_similar_title1 = titles_sorted_indexes1[-3]
third_most_similar_title1 = titles_sorted_indexes1[-4]

most_similar_title2 = titles_sorted_indexes2[-2]
second_most_similar_title2 = titles_sorted_indexes2[-3]
third_most_similar_title2 = titles_sorted_indexes2[-4]

most_similar_title3 = titles_sorted_indexes3[-2]
second_most_similar_title3 = titles_sorted_indexes3[-3]
third_most_similar_title3 = titles_sorted_indexes3[-4]

In [68]:
title_frames1 = [df_titles.iloc[[most_similar_title1]], df_titles.iloc[[second_most_similar_title1]], df_titles.iloc[[third_most_similar_title1]]]
result1 = pd.concat(title_frames1)
result1

,title
3138,sandalinha milicia e sucesso nos morros do rio
1629,macaranduba e o novo chefe de policia do rio
1012,em homenagem ao filho gay edir macedo dispensa homossexuais de pagarem dizimo na sua igreja


In [69]:
title_frames2 = [df_titles.iloc[[most_similar_title2]], df_titles.iloc[[second_most_similar_title2]], df_titles.iloc[[third_most_similar_title2]]]
result2 = pd.concat(title_frames2)
result2

,title
3780,cego entra com processo contra silvio santos ele falou pra mim se enxerga
1576,ansiosa pelo fgts dilma agradece a temer
657,apos protestos contra corrupcao colocarem pmdb no poder brasileiros queimam pneus contra poluicao


In [70]:
title_frames3 = [df_titles.iloc[[most_similar_title3]], df_titles.iloc[[second_most_similar_title3]], df_titles.iloc[[third_most_similar_title3]]]
result3 = pd.concat(title_frames3)
result3

,title
796,marco feliciano lanca projeto de lei contra exame da prostata
1884,orgulho hetero movimento quer revisao historica somos oriundos dos hetero sapiens
2647,omo vai patrocinar marcha do orgulho hetero


### K-NN: título como entrada e textos para cálculo de similaridades

In [71]:
df_texts = df.drop(columns=['title', 'label'])

pd.options.display.max_colwidth = 100
text_series = df_texts['text']
text_series

0       a familia guimaraes passou a noite de natal perplexa.  genival o tiozao do pave deles desde 1985...
1       uma menina de 11 anos apedrejada ao sair de uma festa de candomble no ultimo domingo no rio de j...
2       insatisfeitas com seus partidos com as siglas dos partidos com os politicos com as tardes ensola...
3       o instituto nupal nucleo de pesquisas da america latina fez uma pesquisa que mostrou ideias que ...
4       assassinatos sequestros mortes violentas.  nenhuma noticia parece abalar a apresentadora do cida...
                                                       ...                                                 
4369    ferias de janeiro acabaram carnaval acabou.  o novo ano novo iniciado na segundafeira poscarnava...
4370    sextafeira dia de esperancas no coracao dos solteiros.  mas tudo tem limite.  pelo menos e o que...
4371    a lingua portuguesa passara a ser a primeira do mundo a ter acentuacao para a ironia.  o senado ...
4372    o prefeito fernando 

In [72]:
text_tfidf_matrix = tfidf_vect.transform(text_series) 
text_tfidf_matrix.shape

(4374, 9747)

In [73]:
titles_text_doc_sim1 = np.array(docSimilarities(query_doc_title1, text_tfidf_matrix).toarray())
titles_text_doc_sim2 = np.array(docSimilarities(query_doc_title2, text_tfidf_matrix).toarray())
titles_text_doc_sim3 = np.array(docSimilarities(query_doc_title3, text_tfidf_matrix).toarray())

titles_text_doc_sim1.shape == titles_text_doc_sim2.shape == titles_text_doc_sim3.shape

True

In [74]:
title_text_sorted_indexes1 = np.argsort(titles_text_doc_sim1, axis=None)
title_text_sorted_indexes2 = np.argsort(titles_text_doc_sim2, axis=None)
title_text_sorted_indexes3 = np.argsort(titles_text_doc_sim3, axis=None)

most_similar_title_text1 = title_text_sorted_indexes1[-2]
second_most_similar_title_text1 = title_text_sorted_indexes1[-3]
third_most_similar_title_text1 = title_text_sorted_indexes1[-4]

most_similar_title_text2 = title_text_sorted_indexes2[-2]
second_most_similar_title_text2 = title_text_sorted_indexes2[-3]
third_most_similar_title_text2 = title_text_sorted_indexes2[-4]

most_similar_title_text3 = title_text_sorted_indexes3[-2]
second_most_similar_title_text3 = title_text_sorted_indexes3[-3]
third_most_similar_title_text3 = title_text_sorted_indexes3[-4]

In [75]:
title_text_frames1 = [df_titles.iloc[[most_similar_title_text1]], df_titles.iloc[[second_most_similar_title_text1]], df_titles.iloc[[third_most_similar_title_text1]]]
result_title_text1 = pd.concat(title_text_frames1)
result_title_text1

,title
4181,romero britto se inspirou em pablo do qual e a musica para fazer desenho de papa francisco
2909,mulher anda de joelhos em aparecida pedindo para nao tirar o chefe no amigo oculto
3744,fotos de cabral preso eram mais aguardadas que as de marina ruy barbosa na playboy


In [76]:
title_text_frames2 = [df_titles.iloc[[most_similar_title_text2]], df_titles.iloc[[second_most_similar_title_text2]], df_titles.iloc[[third_most_similar_title_text2]]]
result_title_text2 = pd.concat(title_text_frames2)
result_title_text2

,title
3409,deus pede danos morais apos homem colar adesivo foi deus que me deu em ford ka
2336,deus diz que desde que se tornou evangelico cunha ja adiou a volta de jesus seis vezes
3817,datena declara que ateus nao acreditam em deus e e processado


In [77]:
title_text_frames3 = [df_titles.iloc[[most_similar_title_text3]], df_titles.iloc[[second_most_similar_title_text3]], df_titles.iloc[[third_most_similar_title_text3]]]
result_title_text3 = pd.concat(title_text_frames3)
result_title_text3

,title
3287,filho de edir macedo diz que nao e gay apenas faz exame de prostata diariamente pra se prevenir
2758,marcha do orgulho enrustido ainda e duvida para este fim de semana
796,marco feliciano lanca projeto de lei contra exame da prostata


### K-NN: texto como entrada e para cálculo de similaridades

In [78]:
query_doc_text1 = text_tfidf_matrix.getrow(index1)
query_doc_text2 = text_tfidf_matrix.getrow(index2)
query_doc_text3 = text_tfidf_matrix.getrow(index3)

query_doc_text1.shape == query_doc_text2.shape == query_doc_text3.shape

True

In [79]:
doc_sim_text_text1 = np.array(docSimilarities(query_doc_text1, text_tfidf_matrix).toarray())
doc_sim_text_text2 = np.array(docSimilarities(query_doc_text2, text_tfidf_matrix).toarray())
doc_sim_text_text3 = np.array(docSimilarities(query_doc_text3, text_tfidf_matrix).toarray())

text_sorted_indexes1 = np.argsort(doc_sim_text_text1, axis=None)
text_sorted_indexes2 = np.argsort(doc_sim_text_text2, axis=None)
text_sorted_indexes3 = np.argsort(doc_sim_text_text3, axis=None)

most_similar_text1 = text_sorted_indexes1[-2]
second_most_similar_text1 = text_sorted_indexes1[-3]
third_most_similar_text1 = text_sorted_indexes1[-4]

most_similar_text2 = text_sorted_indexes2[-2]
second_most_similar_text2 = text_sorted_indexes2[-3]
third_most_similar_text2 = text_sorted_indexes2[-4]

most_similar_text3 = text_sorted_indexes3[-2]
second_most_similar_text3 = text_sorted_indexes3[-3]
third_most_similar_text3 = text_sorted_indexes3[-4]



In [80]:
frames_text1 = [df_titles.iloc[[most_similar_text1]], df_titles.iloc[[second_most_similar_text1]], df_titles.iloc[[third_most_similar_text1]]]
result_text1 = pd.concat(frames_text1)
result_text1

,title
3559,estatua de tom jobim usa lentes de contato para nao roubarem os oculos
4128,portugueses tomam comunidade em poder dos indios e cabral instala a primeira upp
1568,crivella anuncia que vai substituir cristo redentor por estatua de edir macedo


In [81]:
frames_text2 = [df_titles.iloc[[most_similar_text2]], df_titles.iloc[[second_most_similar_text2]], df_titles.iloc[[third_most_similar_text2]]]
result_text2 = pd.concat(frames_text2)
result_text2

,title
1269,astrologia veja como venus e presidente retrogrados podem interferir no brasil
3952,movimento separatista do sul apresenta uniforme dois do mascote sulito
4015,caso nao possa concorrer em 2018 lula cogita se candidatar a deus


In [82]:
frames_text3 = [df_titles.iloc[[most_similar_text3]], df_titles.iloc[[second_most_similar_text3]], df_titles.iloc[[third_most_similar_text3]]]
result_text3 = pd.concat(frames_text3)
result_text3

,title
2758,marcha do orgulho enrustido ainda e duvida para este fim de semana
3575,entrevista de mulher de joao doria a jornal causa epidemia de vomito no cafe da manha
1884,orgulho hetero movimento quer revisao historica somos oriundos dos hetero sapiens


## Conclusão e respostas das perguntas

Para analisá-los de forma mais eficiente, vamos concatenar todos os resultados do mesmo título em um mesmo Dataframe:

In [83]:
frames_final1 = [result1, result_title_text1, result_text1]
result_final1 = pd.concat(frames_final1)
result_final1.title.value_counts()

 macaranduba e o novo chefe de policia do rio                                                   1
 em homenagem ao filho gay edir macedo dispensa homossexuais de pagarem dizimo na sua igreja    1
 sandalinha milicia e sucesso nos morros do rio                                                 1
 crivella anuncia que vai substituir cristo redentor por estatua de edir macedo                 1
 fotos de cabral preso eram mais aguardadas que as de marina ruy barbosa na playboy             1
 mulher anda de joelhos em aparecida pedindo para nao tirar o chefe no amigo oculto             1
 portugueses tomam comunidade em poder dos indios e cabral instala a primeira upp               1
 romero britto se inspirou em pablo do qual e a musica para fazer desenho de papa francisco     1
 estatua de tom jobim usa lentes de contato para nao roubarem os oculos                         1
Name: title, dtype: int64

In [84]:
frames_final2 = [result2, result_title_text2, result_text2]
result_final2 = pd.concat(frames_final2)
result_final2.title.value_counts()

 movimento separatista do sul apresenta uniforme dois do mascote sulito                               1
 cego entra com processo contra silvio santos ele falou pra mim se enxerga                            1
 astrologia veja como venus e presidente retrogrados podem interferir no brasil                       1
 ansiosa pelo fgts dilma agradece a temer                                                             1
 deus pede danos morais apos homem colar adesivo foi deus que me deu em ford ka                       1
 caso nao possa concorrer em 2018 lula cogita se candidatar a deus                                    1
 apos protestos contra corrupcao colocarem pmdb no poder brasileiros queimam pneus contra poluicao    1
 datena declara que ateus nao acreditam em deus e e processado                                        1
 deus diz que desde que se tornou evangelico cunha ja adiou a volta de jesus seis vezes               1
Name: title, dtype: int64

In [85]:
frames_final3 = [result3, result_title_text3, result_text3]
result_final3 = pd.concat(frames_final3)
result_final3.title.value_counts()

 marcha do orgulho enrustido ainda e duvida para este fim de semana                                 2
 orgulho hetero movimento quer revisao historica somos oriundos dos hetero sapiens                  2
 marco feliciano lanca projeto de lei contra exame da prostata                                      2
 entrevista de mulher de joao doria a jornal causa epidemia de vomito no cafe da manha              1
 omo vai patrocinar marcha do orgulho hetero                                                        1
 filho de edir macedo diz que nao e gay apenas faz exame de prostata diariamente pra se prevenir    1
Name: title, dtype: int64

### Respostas

1. Os vizinhos são os mesmos em todos os casos?
R: Não são os mesmos, mas houve repetição no terceiro texto (diferentes cálculos chegaram no mesmo resultado de similaridade, em alguns casos).

2. Se não, quais foram diferentes? 
R: Todos os resultados de diferentes estratégias de cálculo no primeiro e segundo texto foram diferentes.

3. Olhando a sátira de entrada, qual das configurações acima (1,2 ou 3) você acha que trouxe os melhores vizinhos e por quê?
R: Na minha opinião, a primeira tática foi a que trouxe os melhores resultados.

4. Nos casos que houve diferença, por que você acha que isso aconteceu?
R: É esperado que estratégias diferentes gerem resultados diferentes, porque o vetor para o cálculo de similaridades muda, assim como o referencial de comparação para medir similaridade (nesse caso, título ou texto). Porém, isso também mostra que nenhuma dessas estratégias é infalível.